In [ ]:
%pip install fastf1
import fastf1
import pandas as pd

In [ ]:
def get_silverstone_results(start_year, end_year):
    all_results = []
    for year in range(start_year, end_year + 1):
        try:
            session = fastf1.get_session(year, 'Silverstone', 'R')
            session.load()
            results = session.results
            results['Year'] = year
            all_results.append(results)
        except Exception as e:
            print(f"Could not load data for year {year}: {e}")
    results_df = pd.concat(all_results, ignore_index=True)
    return results_df[results_df['Status'] == 'Finished' & results_df['Position'] <= 10]

    

In [ ]:
def pitstop_analysis(start_year, end_year):
    pitstop_data = []
    for year in range(start_year, end_year + 1):
        try:
            session = fastf1.get_session(year, 'Silverstone', 'R')
            session.load()
            unique = session.laps
            unique[unique['PitInTime'].notna()] 
            unique[unique]
            unique['PitStopCount'] = unique.groupby('Driver')['PitInTime'].transform('count')
            unique = unique[unique['PitStopCount'] > 0]
            unique['AveragePitStopDuration'] = unique.groupby('Driver')['PitInTime'].transform('mean')
            unique['Year'] = year
            unique = unique.rename(columns={'Driver': 'Abbreviation'})
            unique[['Abbreviation', 'PitStopCount', 'AveragePitStopDuration']].drop_duplicates()      
            pitstop_data.append(unique[['Abbreviation', 'PitStopCount', 'AveragePitStopDuration', 'Year']].drop_duplicates())
        except Exception as e:
            print(f"Could not load data for year {year}: {e}")
    pitstop_df = pd.concat(pitstop_data, ignore_index=True)
    return pitstop_df 

In [ ]:
df_pitstop = pitstop_analysis(2018, 2024)
df_general = get_silverstone_results(2018, 2024)

In [ ]:
final_df = pd.merge(df_general, df_pitstop, on=['Abbreviation', 'Year'], how='left')
final_df